In [1]:
import requests
from bs4 import BeautifulSoup as bs
import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
#Modificamos las cabeceras HTTP y el user agent al detectar la web que accedemos desde python
headers = {
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\
*/*;q=0.8",
"Accept-Encoding": "gzip, deflate, sdch, br",
"Accept-Language": "en-US,en;q=0.8",
"Cache-Control": "no-cache",
"dnt": "1",
"Pragma": "no-cache",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/5\
37.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"
}

# Tomamos la informacion de la web
soup = requests.get("https://socialblade.com/twitter/user/vox_es/monthly", headers=headers)

In [3]:
soup = bs(soup.text, 'lxml')
script_divs = soup.find_all('script', {'type': 'text/javascript'})

In [7]:
# Definimos una funcion para coger solo la parte que nos interesa
def lista_scrap(x, y):
    # Nos quedamos solo con la fila 5 de la lista ya que es donde esta la ifnormacion que queremos
    cadena_ori = script_divs[5]
    # Creamos una lista separando por [ que es por donde empieza nuestra informacion
    cadena_sep = str(cadena_ori).split('[')
    # Selecionamos las filas que queremos segun si es seguidores, menos seguidores...
    lista = cadena_sep[x:y]
    # eliminamos ,]
    lista = [test.replace('],', '').replace('"', '') for test in lista]
    return lista

In [10]:
# Hacemos otra funacion para crear un dataframe con los datos de la anterior funcion
def lista_dataf(x, y):
    df = pd.DataFrame()
    #Usamos la funcion anterior para coger los datos
    lista = lista_scrap(x, y)
    # Señalamos la primera colunma como la fecha y sacamos los datos de la lista
    df['Semana'] = [x.split(',')[0] for x in lista]
    # Señalamos la segunda como los seguidores
    df['Seguidores'] = [x.split(',')[1] for x in lista]
    #Ahora hay que pasar la fecha a un formao mas legible
    #Convertir epoch a fecha normal
    # tenemos que eliminar los tres ultimos ceros para pasorlo
    df['Semana'] = df['Semana'].map(lambda x: str(x)[:-3])
    #pasamos la columna a numeros
    df['Semana'] = pd.to_numeric(df['Semana'])
    #pasamos a formato fecha
    for n in range(len(df)):
        df['Semana'][n] = datetime.datetime.fromtimestamp(df['Semana'][n]).strftime('%d-%m-%Y')
    return df

In [11]:
# Si inspeccionamos los datos vemos de donde a donde van cada uno de los graficos
cadena_ori = script_divs[5]
cadena_sep = str(cadena_ori).split('[')

seguidores_ganados = lista_dataf(3, 182)
seguidos_nuevos = lista_dataf(185, 364)
media = lista_dataf(367, 546)

C:\Users\Dany\Anaconda2\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [12]:
# Juantamos las tres variables
Base_conjunta = pd.DataFrame()
Base_conjunta['Semana'] = seguidores_ganados['Semana']
Base_conjunta['Seguidores'] = seguidores_ganados['Seguidores']
Base_conjunta['Seguidos'] = seguidos_nuevos['Seguidores']
Base_conjunta['Media'] = media['Seguidores']

#Vemos la cabecera de la base
Base_conjunta.head(5)

,Semana,Seguidores,Seguidos,Media
0,22-05-2017,209,5,113
1,29-05-2017,69,7,84
2,05-06-2017,107,9,121
3,12-06-2017,105,2,71
4,19-06-2017,97,8,127


In [13]:
# Guardamos la base en csv
Base_conjunta.to_csv('Vox_Twitter.csv', index=False)